# Term project Data Mining -CSE 5334
# Name : Jugal Pareshbhai Patel
# UTA ID : 1001769143

# Main goal of the project 

***The main goal of the project is to build a classifier which can predict the rating (1-10) from the given reviews or comments.**

# Abstract of the Project 

Here for prediction of rating from given reviews and comment we would compare  Naive Bayes classifier and Support Vector Machine (SVM) .Then after the comparision between these two classifiers , SVM gives better accuracy than Naive Bayes hands down .So now we will use linear SVM classifier for the prediction of ratings. The prediction rating would be in range of 1 - 10 . For Naive Bayes we have used Multinomial Naive Bayes algorithm among three naive bayes algorithm and for SVM we have used linear SVM classifier.

Here are the steps done for achieving the goal of the project which is to predict the rating from given reviews and comments :
1. Load the data from csv file using built in library which is pandas.
2. Pre-Processing of data .
3. Dividing dataset into three parts : Train data , Development data, Test data .
4. Seperate the divided data into two parts : Comments and Ratings
5. Vectorize comment for each divided data and convert the ratings to integer by using our custom function.
6. Now we will use train data to train our SVM model and then we will predict the development and test accuracy.
7. Now we will predict our rating for a single comment using trained SVM model.


In [2]:
import re
import bz2
import pickle
import _pickle as cPickle

import pickle
from sys import path
from os.path import join

import pandas as pd
import numpy as np

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
def loadAndProcessData(filePath):
    data = pd.read_csv(filePath, usecols=['comment', 'rating'])
    data = data.fillna('')
    data = data.sample(frac=1).reset_index(drop=True)

    tempList = list()
    for d in data['comment']:
        d = d.strip().lower()
        #d = re.sub('[\W_]+', '', d, flags = re.UNICODE)
        tempList.append(d)
    data['comment'] = tempList.copy()

    tempList = list()
    for d in data['comment']:
        if d.islower():
            tempList.append(d)
        else:
            tempList.append('')
    data['comment'] = tempList.copy()

    tempList = list()
    for r in data['rating']:
        r = float(r)
        tempList.append(r)
    data['rating'] = tempList.copy()

    data = data[data['comment'].apply(lambda xyz: len(xyz) > 0)]
    data = data[data['rating'].apply(lambda rate: float(rate) >= 1)]
    data = data.sample(frac=1).reset_index(drop=True)

    return data

def getTrainData(data, n):
    data = data.sample(frac=1).reset_index(drop=True)
    lenOfData = data.shape[0]

    endIndex = int(lenOfData * n)

    trainSet = data[: endIndex]
    otherSet = data[endIndex:]

    trainSet = trainSet.sample(frac=1).reset_index(drop=True)
    otherSet = otherSet.sample(frac=1).reset_index(drop=True)

    return [trainSet, otherSet]

def getDevAndTestData(data):
    devSet, testSet = getTrainData(data, 0.5).copy()
    
    if devSet.shape[0] >= testSet.shape[0]:
        return [devSet, testSet]
    else:
        return [testSet, devSet]
    
def generateFile(name, data):
    with bz2.BZ2File(name + '.pbz2', 'w') as f:
        cPickle.dump(data, f)
    f.close()
    
def readFile(name):
    with bz2.BZ2File(name, 'rb') as f:
        data = cPickle.load(f)
    f.close()
    return data

def getVectorizer(data):
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    vectorizer.fit(data)
    generateFile('vectorizer', vectorizer)
    # vectorizerFile = open('vectorizer', 'ab')
    # pickle.dump(vectorizer, vectorizerFile)
    # vectorizerFile.close()

    return vectorizer

def vectorizeData(data, vectorizer):
    data = vectorizer.transform(data)

    return data

def workOnRatings(ratings):
    tempList = list()
    for r in ratings:
        if r%int(r) <= 0.5:
            tempList.append(int(r))
        else:
            tempList.append(int(r + 1))

    return np.asarray(tempList)


def calAccuracy(actual, pred):
    return accuracy_score(actual, pred)*100

In [4]:
pathToFolder = join(path[0], '/kaggle/input/boardgamegeek-reviews')
#/content/drive/My Drive/Colab Notebooks/boardgamegeek-reviews
pathToFile = join(pathToFolder, 'bgg-13m-reviews.csv')

data = loadAndProcessData(pathToFile)
print(data)

trainDataset, tempDataset = getTrainData(data, 0.95).copy()
devDataset, testDataset = getDevAndTestData(tempDataset)

print('\n\n TRAIN DATASET:::\n')
print(trainDataset)
print('\n\n DEVELOPEMENT DATASET:::\n')
print(devDataset)
print('\n\n TEST DATASET:::')
print(testDataset)


trainComments, trainRatings = [trainDataset['comment'], trainDataset['rating']].copy()
devComments, devRatings = [devDataset['comment'], devDataset['rating']].copy()
testComments, testRatings = [testDataset['comment'], testDataset['rating']].copy()

try:
  vectorizer = readFile('vectorizer.pbz2')
  # vectorizeFile = open('vectorizer', 'rb')
  # vectorizer = pickle.load(vectorizeFile)
  print("Vectorizer read from file")
except (FileNotFoundError):
  vectorizer = getVectorizer(trainComments)
  print("Vectorizer file was not found... calling function")


trainComments, trainRatings = [vectorizeData(trainComments, vectorizer), workOnRatings(trainRatings)]
devComments, devRatings = [vectorizeData(devComments, vectorizer), workOnRatings(devRatings)]
testComments, testRatings = [vectorizeData(testComments, vectorizer), workOnRatings(testRatings)]




         rating                                            comment
0          10.0                brasil - jogo de xadrez tradicional
1           2.0           the children like it but that's about it
2          10.0  probably shouldn't have gotten rid of this, bu...
3           7.5  quick, fun, light and friendly. excellent game...
4           2.0  the only reason i didn’t give this one a “1” i...
...         ...                                                ...
2592850     7.0  a game that test your vocabulary and ability t...
2592851     6.0  a nice light winston drafter. probably best at...
2592852     6.0  good for it's day.........which was a long tim...
2592853    10.0  i think a solitaire game system can't ever rea...
2592854     8.0                               very dynamic and fun

[2592855 rows x 2 columns]


 TRAIN DATASET:::

         rating                                            comment
0           9.0  my go-to for solo play. lots of strategy, frus...
1           5

# What is Naive Bayes?

Naive Bayes classifier is based on Bayes’ theorem, which is: P(H | x) = P(H) P(x | H) / P(x) P(H|x) is the posterior probability of hypothesis (H or target) given predictor (x or attribute). P(H) is the prior probability of hypothesis P(x|H) is the likelihood which is the probability of predictor given hypothesis. P(x) is the prior probability of predictor. Naive Bayes extends Bayes’ theorem to handle multiple evidences by assuming that each evidence is independent.

Applications of Naive Bayes Algorithm :

* Real time prediction
* Multi class prediction
* Text Classification / Spam Filtering / Sentiment Analysis
* Recommendation Systems

There are 3 main types of Naive Bayes algorithms:

* Gaussian Naive Bayes
* Multinomial Naive Bayes
* Bernoulli Naive Bayes

For our prjoect we have used Multinomial Naive Bayes algorithm for comparision with SVM model.

In [5]:
nbModel = MultinomialNB(alpha = 1)
nbModel.fit(trainComments, trainRatings)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [6]:
nbTrainRatingsResults = nbModel.predict(trainComments)
nbDevRatingsResults = nbModel.predict(devComments)
nbTestRatingsResults = nbModel.predict(testComments)
nbTrainAccuracy = calAccuracy(trainRatings, nbTrainRatingsResults)
nbDevAccuracy = calAccuracy(devRatings, nbDevRatingsResults)
nbTestAccuracy = calAccuracy(testRatings, nbTestRatingsResults)
print("\n\nNB accuracy for train dataset: {:0.5f}".format(nbTrainAccuracy))
print("NB accuracy for development dataset: {:0.5f}".format(nbDevAccuracy))
print("NB accuracy for test dataset(FINAL): {:0.5f}".format(nbTestAccuracy))



NB accuracy for train dataset: 32.25609
NB accuracy for development dataset: 27.68196
NB accuracy for test dataset(FINAL): 27.71633


# What is Support Vector Machine?

The objective of the support vector machine algorithm is to find a hyperplane in an N-dimensional space(N — the number of features) that distinctly classifies the data points. To separate the two classes of data points, there are many possible hyperplanes that could be chosen. Our objective is to find a plane that has the maximum margin, i.e the maximum distance between data points of both classes. Maximizing the margin distance provides some reinforcement so that future data points can be classified with more confidence. Linear SVM is the newest extremely fast machine learning algorithm for solving multiclass classification problems from ultra large data sets.

Advantages:

* SVM works relatively well when there is clear margin of separation between classes.
* SVM is more effective in high dimensional spaces.
* SVM is effective in cases where number of dimensions is greater than the number of samples.
* SVM is relatively memory efficient.

**The consensus for ML researchers and practitioners in almost all cases ,the SVM is better than Naive Bayes.**

# Hyperparameter Tuning:

* Hyperparameter tuning is used to improve the accuracy of SVM classifier.
* Here we have different regularization parameter (c) and we will use that to show relation between accuracy of SVM classifier and whichever c value gives better accuracy , we will select that for improvement in accuracy.

In [7]:
#hyper....
svmModel1 = LinearSVC(C = 0.1)
svmModel1.fit(trainComments, trainRatings)
svmModel3 = LinearSVC(C = 0.3)
svmModel3.fit(trainComments, trainRatings)
svmModel5 = LinearSVC(C = 0.5)
svmModel5.fit(trainComments, trainRatings)
svmModel7 = LinearSVC(C = 0.7)
svmModel7.fit(trainComments, trainRatings)
svmModel = LinearSVC()
svmModel.fit(trainComments, trainRatings)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

# FOR C=0.1

In [8]:
svmTrainRatingsResults = svmModel1.predict(trainComments)
svmDevRatingsResults = svmModel1.predict(devComments)
svmTestRatingsResults = svmModel1.predict(testComments)
svmTrainAccuracy = calAccuracy(trainRatings, svmTrainRatingsResults)
svmDevAccuracy = calAccuracy(devRatings, svmDevRatingsResults)
svmTestAccuracy = calAccuracy(testRatings, svmTestRatingsResults)
print("\n\nSVM accuracy for train dataset: {:0.5f}".format(svmTrainAccuracy))
print("SVM accuracy for development dataset: {:0.5f}".format(svmDevAccuracy))
print("SVM accuracy for test dataset(FINAL): {:0.5f}".format(svmTestAccuracy))



SVM accuracy for train dataset: 57.62484
SVM accuracy for development dataset: 34.83077
SVM accuracy for test dataset(FINAL): 35.08585


# FOR C = 0.3

In [9]:
svmTrainRatingsResults = svmModel3.predict(trainComments)
svmDevRatingsResults = svmModel3.predict(devComments)
svmTestRatingsResults = svmModel3.predict(testComments)
svmTrainAccuracy = calAccuracy(trainRatings, svmTrainRatingsResults)
svmDevAccuracy = calAccuracy(devRatings, svmDevRatingsResults)
svmTestAccuracy = calAccuracy(testRatings, svmTestRatingsResults)
print("\n\nSVM accuracy for train dataset: {:0.5f}".format(svmTrainAccuracy))
print("SVM accuracy for development dataset: {:0.5f}".format(svmDevAccuracy))
print("SVM accuracy for test dataset(FINAL): {:0.5f}".format(svmTestAccuracy))



SVM accuracy for train dataset: 73.23945
SVM accuracy for development dataset: 33.81877
SVM accuracy for test dataset(FINAL): 34.56442


# FOR C= 0.5

In [10]:
svmTrainRatingsResults = svmModel5.predict(trainComments)
svmDevRatingsResults = svmModel5.predict(devComments)
svmTestRatingsResults = svmModel5.predict(testComments)
svmTrainAccuracy = calAccuracy(trainRatings, svmTrainRatingsResults)
svmDevAccuracy = calAccuracy(devRatings, svmDevRatingsResults)
svmTestAccuracy = calAccuracy(testRatings, svmTestRatingsResults)
print("\n\nSVM accuracy for train dataset: {:0.5f}".format(svmTrainAccuracy))
print("SVM accuracy for development dataset: {:0.5f}".format(svmDevAccuracy))
print("SVM accuracy for test dataset(FINAL): {:0.5f}".format(svmTestAccuracy))



SVM accuracy for train dataset: 79.15575
SVM accuracy for development dataset: 33.07673
SVM accuracy for test dataset(FINAL): 33.97356


# FOR C = 0.7

In [11]:
svmTrainRatingsResults = svmModel7.predict(trainComments)
svmDevRatingsResults = svmModel7.predict(devComments)
svmTestRatingsResults = svmModel7.predict(testComments)
svmTrainAccuracy = calAccuracy(trainRatings, svmTrainRatingsResults)
svmDevAccuracy = calAccuracy(devRatings, svmDevRatingsResults)
svmTestAccuracy = calAccuracy(testRatings, svmTestRatingsResults)
print("\n\nSVM accuracy for train dataset: {:0.5f}".format(svmTrainAccuracy))
print("SVM accuracy for development dataset: {:0.5f}".format(svmDevAccuracy))
print("SVM accuracy for test dataset(FINAL): {:0.5f}".format(svmTestAccuracy))



SVM accuracy for train dataset: 82.10747
SVM accuracy for development dataset: 32.68335
SVM accuracy for test dataset(FINAL): 33.53389


# FOR C = 1

In [12]:
svmTrainRatingsResults = svmModel.predict(trainComments)
svmDevRatingsResults = svmModel.predict(devComments)
svmTestRatingsResults = svmModel.predict(testComments)
svmTrainAccuracy = calAccuracy(trainRatings, svmTrainRatingsResults)
svmDevAccuracy = calAccuracy(devRatings, svmDevRatingsResults)
svmTestAccuracy = calAccuracy(testRatings, svmTestRatingsResults)
print("\n\nSVM accuracy for train dataset: {:0.5f}".format(svmTrainAccuracy))
print("SVM accuracy for development dataset: {:0.5f}".format(svmDevAccuracy))
print("SVM accuracy for test dataset(FINAL): {:0.5f}".format(svmTestAccuracy))



SVM accuracy for train dataset: 84.45152
SVM accuracy for development dataset: 32.19740
SVM accuracy for test dataset(FINAL): 32.92143


# So from above steps of hyper-parameter tuning we can see that SVM classifier model's final accuracy is better when c = 0.1 which is 35.08585 .

# So we had got high perfomance in final accuracy which is better than Naive Bayes's accuracy and what that of if SVM would be on itself.

# CHALLENGES :

* ****Dataset is much large which contains more than 13 million of data.
* ****There were many empty reviews, white spaces , non-english words
* ****The review data contains many data in float values which was effecting the final accuracy
* ****Which classifier to select for the best accuracy and predict the rating**


# Actions taken to overcome challenges :

* ****Because of the large dataset we have divided the dataset into different parts.
* ****Conversion of review data which contains float value into integer value.
* ****From viewing the final accuracy of Multinomial Naive Bayes and SVM , I selected SVM classifier for prediction of ratings.  
* ****We have used hyper-parameter tuning to improve the accuracy of SVM classifier.

# CONCLUSION :

# The results shows that from Naive Bayes classifier and SVM classifier, the final accuracy of SVM classifier is way better than Naive Bayes classifier . So now will be using linear SVM classifier for achieving our goal for prediction of rating from given reviews. So from this we can say that for large text dataset after testing multiple times SVM model classifier is way better than other classifier model.

# From default SVM's accuracy which was around 32 % we improved it to around 35 % , which is better than default SVM accuracy and Naive Bayes accuracy.

In [ ]:
try:
  finalSVM = readFile('svmModel.pbz2')
  # vectorizeFile = open('vectorizer', 'rb')
  # vectorizer = pickle.load(vectorizeFile)
  print("Model read from file")
except (FileNotFoundError):
  print("Model file was not found... calling function to create it")
  #finalSVM = generateFile('svmModel', svmModel1) 
  

In [ ]:
while True:
  comment = input("Enter comment here...\n")
  if comment == 'q':
    break
  comment = vectorizer.transform([comment])
  rating = svmModel1.predict(comment)[0]
  print(rating)

Enter comment here...
amazing
10
Enter comment here...
good
7
Enter comment here...
worst
1



# REFERENCES

* https://towardsdatascience.com/svm-implementation-from-scratch-python-2db2fc52e5c2
* https://towardsdatascience.com/vectorization-implementation-in-machine-learning-ca652920c55d
* https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
* https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/
* https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing
* https://www.dataquest.io/blog/machine-learning-python/
